# Heart Disease Prediction via Logistic Regression

First we grab the dataset from Kaggle.

In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "Heart_Disease_Prediction.csv"

df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "neurocipher/heartdisease",
  file_path
)

df.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


The dataset consists of information on a number of patients, followed by the "Heart Disease" column that indicates whether the patient has or doesn't have heart disease.

In [2]:
df.shape

(270, 14)

Dataset consists of 270 patients with 14 pieces of information on each. Next we need to ensure there are no missing values.

In [3]:
df.describe()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,3.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,1.585185,0.670370,4.696296
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,1.940659
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,1.000000,0.000000,3.000000
25%,48.000000,0.000000,3.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,1.000000,0.000000,3.000000
50%,55.000000,1.000000,3.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,2.000000,0.000000,3.000000
75%,61.000000,1.000000,4.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,2.000000,1.000000,7.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,3.000000,3.000000,7.000000


Every column has a `count` of 270, and as `count` only counts non-NaN values, there are no missing values.

In [4]:
labels_str = df["Heart Disease"]
labels_str

0      Presence
1       Absence
2      Presence
3       Absence
4       Absence
         ...   
265     Absence
266     Absence
267     Absence
268     Absence
269    Presence
Name: Heart Disease, Length: 270, dtype: object

This is the target variable, and while it appears to be binary it is a string rather than a binary number, so we need to convert it to that.

In [5]:
labels = labels_str.map({'Absence': 0, 'Presence': 1})
labels.head()

0    1
1    0
2    1
3    0
4    0
Name: Heart Disease, dtype: int64

We now need to ensure "Presence" and "Absence" were indeed the only possible values for "Heart Disease".

In [6]:
labels.unique()

array([1, 0])

This confirms `labels` is binary, so we can move on.

## Logistic Regression with One Input Feature

For this purpose we will choose the feature "Cholesterol". This is because abnormal levels of cholesterol is known to be a fairly well known indicator of cardiovascular issues.

In [7]:
features = df["Cholesterol"]
features.head()

0    322
1    564
2    261
3    263
4    269
Name: Cholesterol, dtype: int64

In [8]:
x = features
y = labels

The features and labels have been chosen. Now, we move to picking the algorithm.

For this purpose, **Logistic Regression** is a good choice. This is because logistic regression is specifically designed for classifying and fitting data into specific containers by transformation using a logistic function.

In [9]:
from sklearn.linear_model import LogisticRegression

One important step that should be performed now is feature scaling. Scaling is necessary to tame outliers, because those outliers would otherwise dominate the decision-making of the model and increase inaccuracy dramatically. For this purpose we will use the `StandardScaler` provided by `sklearn`.

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
import pandas as pd

In [12]:
standardized_data = pd.DataFrame(scaler.fit_transform(x.values.reshape(-1,1)), columns=['cholesterol'])
standardized_data.head()

,cholesterol
0,1.402212
1,6.093004
2,0.219823
3,0.258589
4,0.374890


We now split the data such that some of it is used for training and the rest for testing. We will be using 20% of the data for testing and the rest to train.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_test, y_train, y_test = train_test_split(standardized_data, y, test_size=0.2, shuffle=True, random_state=42)
print("x_train shape", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape (216, 1)
x_test shape: (54, 1)
y_train shape: (216,)
y_test shape: (54,)


Now we need to pick a loss function to tell the algorithm how to perform the learning. We will use the log-loss function, which is the default for scikit-learn (so we don't need to explicitly pick it).

The log loss function is a function that measures the divergence of the predicted probability with the actual labels. The less the output of this function, the better the model. For a hypothetical perfect model, the log loss = 0.

In [15]:
model = LogisticRegression(random_state=42).fit(x_train, y_train)

The model has been trained. We can now see how well it does.

In [16]:
scores = model.score(x_test, y_test)
scores

0.6111111111111112

The score is around 0.611 which indicates the model got 61.1% of its predictions correct.

In [17]:
pred_labels = model.predict(x_test)

In [18]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_labels)
print(cm)

[[28  5]
 [16  5]]


According to the confusion matrix, we can see that the model got 28 positive diagnoses right and 5 negative diagnoses correct, but had 16 false negatives and 5 false positives.

We can find further information by calculating the F1 score, recall, precision and accuracy.

In [19]:
from numpy import mean, std
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [20]:
from sklearn.metrics import classification_report

f1 = f1_score(y_test, pred_labels)
recall = recall_score(y_test, pred_labels)
precision = precision_score(y_test, pred_labels)

f1_avg = mean(f1_score(y_test, pred_labels, average=None))
recall_avg = mean(recall_score(y_test, pred_labels, average=None))
precision_avg = mean(precision_score(y_test, pred_labels, average=None))

f1_sd = std(f1_score(y_test, pred_labels, average=None))
recall_sd = std(recall_score(y_test, pred_labels, average=None))
precision_sd = std(precision_score(y_test, pred_labels, average=None))

print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1_avg)
print('recall_avg\t',recall_avg)
print('precision_avg\t',precision_avg)

print('\nf1_sd:\t\t',f1_sd)
print('recall_sd\t',recall_sd)
print('precision_sd\t',precision_sd)

print('\n',classification_report(y_test, pred_labels))


f1:		 0.3225806451612903
recall		 0.23809523809523808
precision	 0.5

f1_avg:		 0.5249266862170088
recall_avg	 0.5432900432900433
precision_avg	 0.5681818181818181

f1_sd:		 0.2023460410557185
recall_sd	 0.3051948051948052
precision_sd	 0.06818181818181818

               precision    recall  f1-score   support

           0       0.64      0.85      0.73        33
           1       0.50      0.24      0.32        21

    accuracy                           0.61        54
   macro avg       0.57      0.54      0.52        54
weighted avg       0.58      0.61      0.57        54



The precision of 0.5 implies the level of usefulness of the results, which is not particularly high.

The recall of 0.24 is especially poor and implies the results are not very complete; however the class of data in the binary value 0 (negatives) have a significantly higher recall of 0.85.

We can also use cross-validation to get even more accurate numbers without the risk of overfitting on the test data.

In [21]:
from sklearn.model_selection import cross_val_predict

model = LogisticRegression(random_state=42)

cv_pred_labels = cross_val_predict(model, standardized_data, labels, cv=10)

In [22]:
accuracy = accuracy_score(labels, cv_pred_labels)

cm = confusion_matrix(labels, cv_pred_labels)

f1 = f1_score(labels, cv_pred_labels)
recall = recall_score(labels, cv_pred_labels)
precision = precision_score(labels, cv_pred_labels)

f1_avg = mean(f1_score(labels, cv_pred_labels, average=None))
recall_avg = mean(recall_score(labels, cv_pred_labels, average=None))
precision_avg = mean(precision_score(labels, cv_pred_labels, average=None))

f1_sd = std(f1_score(labels, cv_pred_labels, average=None))
recall_sd = std(recall_score(labels, cv_pred_labels, average=None))
precision_sd = std(precision_score(labels, cv_pred_labels, average=None))

print('accuracy:\t', accuracy)

print(cm)

print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1_avg)
print('recall_avg\t',recall_avg)
print('precision_avg\t',precision_avg)

print('\nf1_sd:\t\t',f1_sd)
print('recall_sd\t',recall_sd)
print('precision_sd\t',precision_sd)

print('\n',classification_report(labels, cv_pred_labels))

accuracy:	 0.5407407407407407
[[128  22]
 [102  18]]

f1:		 0.225
recall		 0.15
precision	 0.45

f1_avg:		 0.4493421052631579
recall_avg	 0.5016666666666667
precision_avg	 0.5032608695652174

f1_sd:		 0.2243421052631579
recall_sd	 0.3516666666666667
precision_sd	 0.05326086956521739

               precision    recall  f1-score   support

           0       0.56      0.85      0.67       150
           1       0.45      0.15      0.23       120

    accuracy                           0.54       270
   macro avg       0.50      0.50      0.45       270
weighted avg       0.51      0.54      0.47       270



This provides a final accuracy of 0.54.

Notably, the recall and precision is even lower.

Overall, only using one input feature seems to provide poor results. Theoretically, having multiple features fixes this issue, which we shall test next.

## Logistic Regression with Multiple Input Features

We will now use all the features except for "Heart Disease" which is the label, as the input features.

In [23]:
features = df[df.columns.difference(["Heart Disease"])]
features.head()

,Age,BP,Chest pain type,Cholesterol,EKG results,Exercise angina,FBS over 120,Max HR,Number of vessels fluro,ST depression,Sex,Slope of ST,Thallium
0,70,130,4,322,2,0,0,109,3,2.4,1,2,3
1,67,115,3,564,2,0,0,160,0,1.6,0,2,7
2,57,124,2,261,0,0,0,141,0,0.3,1,1,7
3,64,128,4,263,0,1,0,105,1,0.2,1,2,7
4,74,120,2,269,2,1,0,121,1,0.2,0,1,3


In [24]:
x = features

We now follow similar steps as followed in the single feature form to create the model.

In [25]:
standardized_data = pd.DataFrame(scaler.fit_transform(x), columns=["age", "bp", "chest pain type", "cholesterol",
                                                                   "ekg results", "exercise angina", "fbs over 120",
                                                                   "max hr", "number of vessels fluro", "st depression",
                                                                   "sex", "slope of st", "thallium"])
standardized_data.head()

,age,bp,chest pain type,cholesterol,ekg results,exercise angina,fbs over 120,max hr,number of vessels fluro,st depression,sex,slope of st,thallium
0,1.712094,-0.075410,0.870928,1.402212,0.981664,-0.701222,-0.417029,-1.759208,2.472682,1.181012,0.689500,0.676419,-0.875706
1,1.382140,-0.916759,-0.183559,6.093004,0.981664,-0.701222,-0.417029,0.446409,-0.711535,0.481153,-1.450327,0.676419,1.189277
2,0.282294,-0.411950,-1.238045,0.219823,-1.026285,-0.701222,-0.417029,-0.375291,-0.711535,-0.656118,0.689500,-0.954234,1.189277
3,1.052186,-0.187590,0.870928,0.258589,-1.026285,1.426081,-0.417029,-1.932198,0.349871,-0.743600,0.689500,0.676419,1.189277
4,2.152032,-0.636310,-1.238045,0.374890,0.981664,1.426081,-0.417029,-1.240239,0.349871,-0.743600,-1.450327,-0.954234,-0.875706


In [26]:
x_train, x_test, y_train, y_test = train_test_split(standardized_data, y, test_size=0.2, shuffle=True, random_state=42)
print("x_train shape", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape (216, 13)
x_test shape: (54, 13)
y_train shape: (216,)
y_test shape: (54,)


In [27]:
model = LogisticRegression(random_state=42).fit(x_train, y_train)

The model is now ready. We can now test to see how well this one does.

In [28]:
scores = model.score(x_test, y_test)
scores

0.9074074074074074

Immediately, we can see that the score has shot up to 0.907 from 0.611, indicating this model is significantly better performing.

In [29]:
pred_labels = model.predict(x_test)

In [30]:
cm = confusion_matrix(y_test, pred_labels)
print(cm)

[[31  2]
 [ 3 18]]


The confusion matrix tells a similar story; there are very few false positives and false negatives and a lot of correct predictions.

Now we prepare the full classification report.

In [31]:
from sklearn.metrics import classification_report

f1 = f1_score(y_test, pred_labels)
recall = recall_score(y_test, pred_labels)
precision = precision_score(y_test, pred_labels)

f1_avg = mean(f1_score(y_test, pred_labels, average=None))
recall_avg = mean(recall_score(y_test, pred_labels, average=None))
precision_avg = mean(precision_score(y_test, pred_labels, average=None))

f1_sd = std(f1_score(y_test, pred_labels, average=None))
recall_sd = std(recall_score(y_test, pred_labels, average=None))
precision_sd = std(precision_score(y_test, pred_labels, average=None))

print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1_avg)
print('recall_avg\t',recall_avg)
print('precision_avg\t',precision_avg)

print('\nf1_sd:\t\t',f1_sd)
print('recall_sd\t',recall_sd)
print('precision_sd\t',precision_sd)

print('\n',classification_report(y_test, pred_labels))


f1:		 0.8780487804878049
recall		 0.8571428571428571
precision	 0.9

f1_avg:		 0.9017109574080815
recall_avg	 0.8982683982683983
precision_avg	 0.9058823529411765

f1_sd:		 0.023662176920276667
recall_sd	 0.041125541125541176
precision_sd	 0.00588235294117645

               precision    recall  f1-score   support

           0       0.91      0.94      0.93        33
           1       0.90      0.86      0.88        21

    accuracy                           0.91        54
   macro avg       0.91      0.90      0.90        54
weighted avg       0.91      0.91      0.91        54



The final accuracy here is 0.91, far higher than when using only one input feature.

The precision and recall are also significantly higher at 0.9 and 0.857 respectively, indicating that the results are both quite useful and also have a high level of completeness.